## Code init

#### Imports

In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
# from html_table_parser.parser import HTMLTableParser
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
# import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


In [2]:
data = pd.read_csv('combinedata.csv')
options = pd.read_csv('option_df.csv')

### **Simulation starting code**

Pseudocode format for mass simulation of individual date-strike rebalance. Essentially, we will:
1. Pull in calls_df and puts_df
2. For dates in both dataframes where is_present is true for both, perform additional analysis
3. Align calls/puts on that day, specify the contract
4. For the contract in the main dataframe, find the period of info until the end date, inclusive, for that strike, contract, and expiry
5. For each day, calculate the delta sum as the sum of the call delta and put delta
6. Based on this, calculate how many Shares we must hold on that day (+ or -) and use this to calculate our share movement daily
7. Calculate the shareprice times shares as sharevalue
8. Calculate PL of day-to-day as change in shares times diff in shareprice
9. Calculate current available closing price of the option straddle, which will also be used to add to the total PL with open positions (ie. price based on the closest book level, should we choose to close the position)
10. On the end date, close ALL positions

We will save each dataframe as an item in a variable () then store it under a same-directory data folder, naming each simulation after the date it was opened and whether we were long or short gamma.

Later we will also simulate:
- integer shares ie. rounded to the nearest
- trading cost as a variable %
- scaling trading algorithms? like how we talked about in class

For now, let's proceed in steps.

In [3]:
data['exdate'] = pd.to_datetime(data['exdate'])
options['exdate'] = pd.to_datetime(options['exdate'])


data['exdate_str'] = data['exdate'].dt.strftime('%Y%m%d')  
data['strikeID'] = data['exdate_str'] + '_' + data['strike_price'].astype(str)  
data.drop(columns=['exdate_str'], inplace=True)  

options['exdate_str'] = options['exdate'].dt.strftime('%Y%m%d')  
options['strikeID'] = options['exdate_str'] + '_' + options['strike_price'].astype(str)  
options.drop(columns=['exdate_str'], inplace=True)  

In [4]:
display(data)
display(options)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume,strikeID
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,20180103_235.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,20180103_240.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,20180103_242.5
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,20180103_245.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0,20180103_247.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,20250321_600.0
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,20250321_605.0
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,20250321_610.0
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,396.26,391.240559,390.285185,96141367.0,20250321_615.0


,date,exdate,strike_price,close_date,close,adj_open,adj_close,adj_volume,TTE,last_date_c,...,volume_p,open_interest_p,impl_volatility_p,delta_p,gamma_p,vega_p,theta_p,expiry_indicator_p,is_present_p,strikeID
0,2018-01-02,2018-01-31,269.0,2018-01-23,268.77,242.053393,242.893856,86655749.0,29,2018-01-02,...,198.0,33.0,0.069577,-0.496183,0.077797,30.11151,-11.46749,w,True,20180131_269.0
1,2018-01-03,2018-02-02,270.0,2018-01-24,270.47,243.065564,244.430187,90070416.0,30,2018-01-03,...,1060.0,330.0,0.069568,-0.442522,0.074853,30.52431,-11.33461,w,True,20180202_270.0
2,2018-01-04,2018-02-02,272.0,2018-01-25,271.61,245.089905,245.460432,80595402.0,29,2018-01-04,...,416.0,304.0,0.068201,-0.508652,0.078707,30.40449,-11.28616,w,True,20180202_272.0
3,2018-01-08,2018-02-07,275.0,2018-01-29,273.92,246.996762,247.548034,57288979.0,30,2018-01-08,...,82.0,269.0,0.067924,-0.560465,0.076891,30.79770,-10.80407,w,True,20180207_275.0
4,2018-01-09,2018-02-07,275.0,2018-01-30,274.54,247.981821,248.108343,57253957.0,29,2018-01-09,...,453.0,229.0,0.076654,-0.510832,0.069026,30.74049,-12.99855,w,True,20180207_275.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,2023-02-22,2023-03-24,399.0,2023-02-28,398.54,393.496030,392.530807,83574386.0,30,2023-02-22,...,2678.0,1317.0,0.201626,-0.506205,0.017387,45.30915,-47.38424,w,True,20230324_399.0
1136,2023-02-23,2023-03-24,401.0,2023-02-28,400.66,395.505271,394.618841,95842681.0,29,2023-02-23,...,1233.0,1566.0,0.186139,-0.507514,0.019064,44.77837,-44.24160,w,True,20230324_401.0
1137,2023-02-24,2023-03-24,396.0,2023-02-28,396.38,389.457850,390.403375,108144866.0,28,2023-02-24,...,3156.0,1086.0,0.198990,-0.494048,0.018342,43.53449,-48.60954,w,True,20230324_396.0
1138,2023-02-27,2023-03-31,398.0,2023-02-28,397.73,393.840753,391.733020,80318244.0,32,2023-02-27,...,1568.0,4088.0,0.188579,-0.505251,0.018253,46.60089,-42.60824,m,True,20230331_398.0


In [5]:
def create_simulations(options_subset, data, dropna_greeks=False):
    simulations = {}

    for index, row in options_subset.iterrows():
        strikeID = row['exdate'].strftime('%Y%m%d') + '_' + str(row['strike_price'])
        mask = (data['strikeID'] == strikeID) & (data['date'] >= row['date']) & (data['date'] <= row['close_date'])
        temp_df = data[mask].sort_values(by=['date', 'cp_flag'])

        shared_cols = ['date', 'exdate', 'strike_price', 'expiry_indicator', 'close', 'adj_open', 'adj_close', 'adj_volume', 'strikeID']
        greeks_cols = ['impl_volatility', 'delta', 'gamma', 'vega', 'theta']
        call_specific_cols = ['cp_flag', 'best_bid', 'best_offer', 'volume', 'open_interest'] + greeks_cols
        put_specific_cols = call_specific_cols

        calls = temp_df[temp_df['cp_flag'] == 'C'][shared_cols + call_specific_cols].rename(columns={col: col + '_c' for col in call_specific_cols})
        puts = temp_df[temp_df['cp_flag'] == 'P'][shared_cols + put_specific_cols].rename(columns={col: col + '_p' for col in put_specific_cols})

        merged_df = pd.merge(calls, puts, on=shared_cols, how='outer')

        if dropna_greeks:
            greeks_cols_c = [col + '_c' for col in greeks_cols]
            greeks_cols_p = [col + '_p' for col in greeks_cols]
            merged_df = merged_df.dropna(subset=greeks_cols_c + greeks_cols_p, how='any')

        merged_df['delta_sum'] = merged_df['delta_c'].fillna(0) + merged_df['delta_p'].fillna(0)
        merged_df['shares_held'] = -1 * merged_df['delta_sum']

        merged_df = merged_df.sort_values(by='date')
        merged_df['sharechange'] = merged_df['shares_held'].diff()

        simulations[row['date'].strftime('%Y-%m-%d')] = merged_df

    return simulations

In [6]:
options['date'] = pd.to_datetime(options['date'])
data['date'] = pd.to_datetime(data['date'])

In [16]:
options_subset = options.head(2)
simulations1 = create_simulations(options_subset, data, dropna_greeks=True)

In [8]:
for key, df in list(simulations1.items())[:2]: 
    print(f"DataFrame for {key}:")
    print(df.columns)
    display(df)  
    print("\n")

DataFrame for 2018-01-02:
Index(['date', 'exdate', 'strike_price', 'expiry_indicator', 'close',
       'adj_open', 'adj_close', 'adj_volume', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'volume_c', 'open_interest_c',
       'impl_volatility_c', 'delta_c', 'gamma_c', 'vega_c', 'theta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'volume_p',
       'open_interest_p', 'impl_volatility_p', 'delta_p', 'gamma_p', 'vega_p',
       'theta_p', 'delta_sum', 'shares_held', 'sharechange'],
      dtype='object')


,date,exdate,strike_price,expiry_indicator,close,adj_open,adj_close,adj_volume,strikeID,cp_flag_c,...,volume_p,open_interest_p,impl_volatility_p,delta_p,gamma_p,vega_p,theta_p,delta_sum,shares_held,sharechange
0,2018-01-02,2018-01-31,269.0,w,268.77,242.053393,242.893856,86655749.0,20180131_269.0,C,...,198.0,33.0,0.069577,-0.496183,0.077797,30.111510,-11.467490,0.015848,-0.015848,NaN
1,2018-01-03,2018-01-31,269.0,w,270.47,243.065564,244.430187,90070416.0,20180131_269.0,C,...,537.0,142.0,0.072179,-0.370678,0.070932,28.250190,-11.921280,0.259359,-0.259359,-0.243511
2,2018-01-04,2018-01-31,269.0,w,271.61,245.089905,245.460432,80595402.0,20180131_269.0,C,...,411.0,408.0,0.077402,-0.303227,0.061720,25.781000,-12.327160,0.391860,-0.391860,-0.132501
3,2018-01-05,2018-01-31,269.0,w,273.42,246.273783,247.096172,83468662.0,20180131_269.0,C,...,1271.0,733.0,0.084433,-0.218306,0.048138,21.481350,-11.877760,0.560376,-0.560376,-0.168516
4,2018-01-08,2018-01-31,269.0,w,273.92,246.996762,247.548034,57288979.0,20180131_269.0,C,...,918.0,1482.0,0.085578,-0.185301,0.045619,18.362690,-11.728420,0.613018,-0.613018,-0.052642
5,2018-01-09,2018-01-31,269.0,w,274.54,247.981821,248.108343,57253957.0,20180131_269.0,C,...,123.0,2265.0,0.094566,-0.177220,0.040922,17.498160,-13.020330,0.646666,-0.646666,-0.033648
6,2018-01-10,2018-01-31,269.0,w,274.12,247.331140,247.728779,69499524.0,20180131_269.0,C,...,9029.0,2307.0,0.091961,-0.183642,0.044112,17.468620,-13.232750,0.626662,-0.626662,0.020004
7,2018-01-11,2018-01-31,269.0,w,276.12,248.298125,249.536226,62306557.0,20180131_269.0,C,...,447.0,9384.0,0.099933,-0.122711,0.031545,13.118760,-11.482330,0.780802,-0.780802,-0.154140
8,2018-01-12,2018-01-31,269.0,w,277.92,249.807344,251.162929,90789911.0,20180131_269.0,C,...,4673.0,9352.0,0.110215,-0.090212,0.023322,10.328780,-10.552090,0.840416,-0.840416,-0.059614
9,2018-01-16,2018-01-31,269.0,w,276.97,252.455254,250.304392,106555142.0,20180131_269.0,C,...,2454.0,11564.0,0.125662,-0.118707,0.028183,11.123210,-16.564950,0.747087,-0.747087,0.093329




DataFrame for 2018-01-03:
Index(['date', 'exdate', 'strike_price', 'expiry_indicator', 'close',
       'adj_open', 'adj_close', 'adj_volume', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'volume_c', 'open_interest_c',
       'impl_volatility_c', 'delta_c', 'gamma_c', 'vega_c', 'theta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'volume_p',
       'open_interest_p', 'impl_volatility_p', 'delta_p', 'gamma_p', 'vega_p',
       'theta_p', 'delta_sum', 'shares_held', 'sharechange'],
      dtype='object')


,date,exdate,strike_price,expiry_indicator,close,adj_open,adj_close,adj_volume,strikeID,cp_flag_c,...,volume_p,open_interest_p,impl_volatility_p,delta_p,gamma_p,vega_p,theta_p,delta_sum,shares_held,sharechange
0,2018-01-03,2018-02-02,270.0,w,270.47,243.065564,244.430187,90070416.0,20180202_270.0,C,...,1060.0,330.0,0.069568,-0.442522,0.074853,30.524310,-11.33461,0.119696,-0.119696,NaN
1,2018-01-04,2018-02-02,270.0,w,271.61,245.089905,245.460432,80595402.0,20180202_270.0,C,...,3453.0,978.0,0.074706,-0.366922,0.066809,28.778260,-12.15496,0.265768,-0.265768,-0.146072
2,2018-01-05,2018-02-02,270.0,w,273.42,246.273783,247.096172,83468662.0,20180202_270.0,C,...,9871.0,3851.0,0.081783,-0.270119,0.053852,25.017800,-12.28042,0.457539,-0.457539,-0.191771
3,2018-01-08,2018-02-02,270.0,w,273.92,246.996762,247.548034,57288979.0,20180202_270.0,C,...,5628.0,13217.0,0.082307,-0.234696,0.052391,22.018190,-12.28724,0.514544,-0.514544,-0.057005
4,2018-01-09,2018-02-02,270.0,w,274.54,247.981821,248.108343,57253957.0,20180202_270.0,C,...,1564.0,14085.0,0.091492,-0.223283,0.046611,21.022120,-13.72542,0.549888,-0.549888,-0.035344
5,2018-01-10,2018-02-02,270.0,w,274.12,247.331140,247.728779,69499524.0,20180202_270.0,C,...,1112.0,14380.0,0.089866,-0.235707,0.050050,21.179100,-14.17493,0.523363,-0.523363,0.026525
6,2018-01-11,2018-02-02,270.0,w,276.12,248.298125,249.536226,62306557.0,20180202_270.0,C,...,5496.0,14105.0,0.097319,-0.162272,0.037349,16.636580,-12.78389,0.692574,-0.692574,-0.169211
7,2018-01-12,2018-02-02,270.0,w,277.92,249.807344,251.162929,90789911.0,20180202_270.0,C,...,1329.0,17847.0,0.107499,-0.121856,0.028289,13.466870,-12.09646,0.768158,-0.768158,-0.075584
8,2018-01-16,2018-02-02,270.0,w,276.97,252.455254,250.304392,106555142.0,20180202_270.0,C,...,2313.0,18201.0,0.121851,-0.157003,0.033063,14.348100,-18.15760,0.670163,-0.670163,0.097995
9,2018-01-17,2018-02-02,270.0,w,279.61,251.262339,252.690223,113201396.0,20180202_270.0,C,...,473.0,18543.0,0.135023,-0.101488,0.022474,10.380940,-15.57961,0.802791,-0.802791,-0.132628


In [17]:
def calculate_realized_PL(df, long_op=True):
    df = df.reset_index(drop=True)  
    df['stock_pos'] = df['shares_held'] if long_op else -df['shares_held']
    df['avg_cost'] = np.nan
    df['realized_PL'] = 0.0
    df['option_PL'] = 0.0
    df['stock_PL'] = 0.0  

    initial_option_value = df.loc[0, 'best_offer_c'] + df.loc[0, 'best_offer_p'] if long_op else -df.loc[0, 'best_bid_c'] - df.loc[0, 'best_bid_p']

    for i in range(len(df)):
        close_price = df.loc[i, 'close']
        df.loc[i, 'option_PL'] = (df.loc[i, 'best_bid_c'] + df.loc[i, 'best_bid_p'] - initial_option_value) if long_op else (-df.loc[i, 'best_offer_c'] - df.loc[i, 'best_offer_p'] - initial_option_value)

        if i == 0:
            df.loc[i, 'avg_cost'] = close_price
            continue

        prev_pos = df.loc[i - 1, 'stock_pos']
        current_pos = df.loc[i, 'stock_pos']
        pos_change = current_pos - prev_pos

        if not pd.isna(df.loc[i - 1, 'avg_cost']):
            df.loc[i, 'stock_PL'] = df.loc[i - 1, 'realized_PL'] + (close_price - df.loc[i - 1, 'avg_cost']) * prev_pos

        daily_option_value = df.loc[i, 'best_bid_c'] + df.loc[i, 'best_bid_p'] if long_op else -df.loc[i, 'best_offer_c'] - df.loc[i, 'best_offer_p']
        df.loc[i, 'option_PL'] = daily_option_value - initial_option_value

        if pos_change != 0:
            if np.sign(pos_change) == np.sign(prev_pos) or prev_pos == 0:
                total_shares = abs(prev_pos) + abs(pos_change)
                total_cost = df.loc[i - 1, 'avg_cost'] * abs(prev_pos) + close_price * abs(pos_change)
                df.loc[i, 'avg_cost'] = total_cost / total_shares if total_shares != 0 else close_price
            else:
                closed_shares = min(abs(prev_pos), abs(pos_change))
                realized_PL_this_step = (close_price - df.loc[i - 1, 'avg_cost']) * closed_shares * np.sign(prev_pos)
                df.loc[i, 'realized_PL'] = df.loc[i - 1, 'realized_PL'] + realized_PL_this_step
                if abs(pos_change) > abs(prev_pos):
                    excess_shares = abs(pos_change) - abs(prev_pos)
                    df.loc[i, 'avg_cost'] = close_price
                    df.loc[i, 'stock_pos'] = excess_shares * np.sign(pos_change)
                else:
                    df.loc[i, 'avg_cost'] = np.nan
        else:
            df.loc[i, 'avg_cost'] = df.loc[i - 1, 'avg_cost']
            df.loc[i, 'stock_pos'] = prev_pos

        df['avg_cost'].ffill(inplace=True)

    final_row_index = len(df) - 1
    final_pos = df.loc[final_row_index, 'stock_pos']
    final_close_price = df.loc[final_row_index, 'close']
    final_avg_cost = df.loc[final_row_index, 'avg_cost']
    final_realized_PL = (final_close_price - final_avg_cost) * final_pos
    df.loc[final_row_index, 'realized_PL'] += final_realized_PL
    df.loc[final_row_index, 'stock_PL'] = df.loc[final_row_index, 'realized_PL']
    df.loc[final_row_index, 'stock_pos'] = 0

    return df


In [18]:
for date, df in simulations1.items():
    simulations1[date] = calculate_realized_PL(df, long_op=True) 

In [19]:
for key, df in list(simulations1.items())[:2]: 
    print(f"DataFrame for {key}:")
    print(df.columns)
    display(df)  
    print("\n")

DataFrame for 2018-01-02:
Index(['date', 'exdate', 'strike_price', 'expiry_indicator', 'close',
       'adj_open', 'adj_close', 'adj_volume', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'volume_c', 'open_interest_c',
       'impl_volatility_c', 'delta_c', 'gamma_c', 'vega_c', 'theta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'volume_p',
       'open_interest_p', 'impl_volatility_p', 'delta_p', 'gamma_p', 'vega_p',
       'theta_p', 'delta_sum', 'shares_held', 'sharechange', 'stock_pos',
       'avg_cost', 'realized_PL', 'option_PL', 'stock_PL'],
      dtype='object')


,date,exdate,strike_price,expiry_indicator,close,adj_open,adj_close,adj_volume,strikeID,cp_flag_c,...,vega_p,theta_p,delta_sum,shares_held,sharechange,stock_pos,avg_cost,realized_PL,option_PL,stock_PL
0,2018-01-02,2018-01-31,269.0,w,268.77,242.053393,242.893856,86655749.0,20180131_269.0,C,...,30.111510,-11.467490,0.015848,-0.015848,NaN,-0.015848,268.770000,0.000000,-0.10,0.000000
1,2018-01-03,2018-01-31,269.0,w,270.47,243.065564,244.430187,90070416.0,20180131_269.0,C,...,28.250190,-11.921280,0.259359,-0.259359,-0.243511,-0.259359,270.366122,0.000000,0.32,-0.026942
2,2018-01-04,2018-01-31,269.0,w,271.61,245.089905,245.460432,80595402.0,20180131_269.0,C,...,25.781000,-12.327160,0.391860,-0.391860,-0.132501,-0.391860,270.786719,0.000000,0.90,-0.322611
3,2018-01-05,2018-01-31,269.0,w,273.42,246.273783,247.096172,83468662.0,20180131_269.0,C,...,21.481350,-11.877760,0.560376,-0.560376,-0.168516,-0.560376,271.578598,0.000000,2.00,-1.031877
4,2018-01-08,2018-01-31,269.0,w,273.92,246.996762,247.548034,57288979.0,20180131_269.0,C,...,18.362690,-11.728420,0.613018,-0.613018,-0.052642,-0.613018,271.779662,0.000000,2.19,-1.312065
5,2018-01-09,2018-01-31,269.0,w,274.54,247.981821,248.108343,57253957.0,20180131_269.0,C,...,17.498160,-13.020330,0.646666,-0.646666,-0.033648,-0.646666,271.923291,0.000000,2.67,-1.692137
6,2018-01-10,2018-01-31,269.0,w,274.12,247.331140,247.728779,69499524.0,20180131_269.0,C,...,17.468620,-13.232750,0.626662,-0.626662,0.020004,-0.626662,271.923291,-0.043943,2.29,-1.420537
7,2018-01-11,2018-01-31,269.0,w,276.12,248.298125,249.536226,62306557.0,20180131_269.0,C,...,13.118760,-11.482330,0.780802,-0.780802,-0.154140,-0.780802,272.751774,0.000000,3.68,-2.673861
8,2018-01-12,2018-01-31,269.0,w,277.92,249.807344,251.162929,90789911.0,20180131_269.0,C,...,10.328780,-10.552090,0.840416,-0.840416,-0.059614,-0.840416,273.118376,0.000000,5.29,-4.035362
9,2018-01-16,2018-01-31,269.0,w,276.97,252.455254,250.304392,106555142.0,20180131_269.0,C,...,11.123210,-16.564950,0.747087,-0.747087,0.093329,-0.747087,273.118376,-0.359468,4.73,-3.236966




DataFrame for 2018-01-03:
Index(['date', 'exdate', 'strike_price', 'expiry_indicator', 'close',
       'adj_open', 'adj_close', 'adj_volume', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'volume_c', 'open_interest_c',
       'impl_volatility_c', 'delta_c', 'gamma_c', 'vega_c', 'theta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'volume_p',
       'open_interest_p', 'impl_volatility_p', 'delta_p', 'gamma_p', 'vega_p',
       'theta_p', 'delta_sum', 'shares_held', 'sharechange', 'stock_pos',
       'avg_cost', 'realized_PL', 'option_PL', 'stock_PL'],
      dtype='object')


,date,exdate,strike_price,expiry_indicator,close,adj_open,adj_close,adj_volume,strikeID,cp_flag_c,...,vega_p,theta_p,delta_sum,shares_held,sharechange,stock_pos,avg_cost,realized_PL,option_PL,stock_PL
0,2018-01-03,2018-02-02,270.0,w,270.47,243.065564,244.430187,90070416.0,20180202_270.0,C,...,30.524310,-11.33461,0.119696,-0.119696,NaN,-0.119696,270.470000,0.000000,-0.07,0.000000
1,2018-01-04,2018-02-02,270.0,w,271.61,245.089905,245.460432,80595402.0,20180202_270.0,C,...,28.778260,-12.15496,0.265768,-0.265768,-0.146072,-0.265768,271.096569,0.000000,0.39,-0.136453
2,2018-01-05,2018-02-02,270.0,w,273.42,246.273783,247.096172,83468662.0,20180202_270.0,C,...,25.017800,-12.28042,0.457539,-0.457539,-0.191771,-0.457539,272.070402,0.000000,1.33,-0.617494
3,2018-01-08,2018-02-02,270.0,w,273.92,246.996762,247.548034,57288979.0,20180202_270.0,C,...,22.018190,-12.28724,0.514544,-0.514544,-0.057005,-0.514544,272.275315,0.000000,1.49,-0.846263
4,2018-01-09,2018-02-02,270.0,w,274.54,247.981821,248.108343,57253957.0,20180202_270.0,C,...,21.022120,-13.72542,0.549888,-0.549888,-0.035344,-0.549888,272.420877,0.000000,2.01,-1.165280
5,2018-01-10,2018-02-02,270.0,w,274.12,247.331140,247.728779,69499524.0,20180202_270.0,C,...,21.179100,-14.17493,0.523363,-0.523363,0.026525,-0.523363,272.420877,-0.045069,1.63,-0.934327
6,2018-01-11,2018-02-02,270.0,w,276.12,248.298125,249.536226,62306557.0,20180202_270.0,C,...,16.636580,-12.78389,0.692574,-0.692574,-0.169211,-0.692574,273.324654,0.000000,2.83,-1.981053
7,2018-01-12,2018-02-02,270.0,w,277.92,249.807344,251.162929,90789911.0,20180202_270.0,C,...,13.466870,-12.09646,0.768158,-0.768158,-0.075584,-0.768158,273.776819,0.000000,4.39,-3.182617
8,2018-01-16,2018-02-02,270.0,w,276.97,252.455254,250.304392,106555142.0,20180202_270.0,C,...,14.348100,-18.15760,0.670163,-0.670163,0.097995,-0.670163,273.776819,-0.312916,3.93,-2.452867
9,2018-01-17,2018-02-02,270.0,w,279.61,251.262339,252.690223,113201396.0,20180202_270.0,C,...,10.380940,-15.57961,0.802791,-0.802791,-0.132628,-0.802791,274.740511,0.000000,5.92,-4.222098


# Simulate and Save

Full Simulation and saving to CSV of all possible position opens.

In [20]:
simulations = create_simulations(options, data, dropna_greeks=True)

simulations_long = {date: calculate_realized_PL(df.copy(), long_op=True) for date, df in simulations.items()}
simulations_short = {date: calculate_realized_PL(df.copy(), long_op=False) for date, df in simulations.items()}

In [21]:
os.makedirs('simdata', exist_ok=True)

for date, df_long in simulations_long.items():
    csv_path_long = f'simdata/simulation_long_{date}.csv'
    df_long.to_csv(csv_path_long, index=False)
    
for date, df_short in simulations_short.items():
    csv_path_short = f'simdata/simulation_short_{date}.csv'
    df_short.to_csv(csv_path_short, index=False)
